In [2]:
import random 
import math
import scipy
import numpy as np
import matplotlib.pyplot as plt
import library_data_science as lds

# Overfitting

# Test of the Selected Model

* We can use **linear regression** to fit a curve to the data.

* That curve is **a model of the data** that can be used to predict the values associated with independent values we haven't seen.

* $R^2$ is used to **evaluate model**.

* Choose complexity of model based on:

    - **theory** about structure of data,

    - **cross validation**,
    
    - **simplicity**.

# Leave-One-Out Cross Validation (LOOCV)

# K-Fold Cross Validation

# Repeated Random Sampling Cross Validation (RRSCV)

# Comparison of Cross Validation Methods